# notebook to retrieve articles from URLs and return clean dataframe

In [ ]:
import re
import pandas as pd
from urllib.request import urlopen
from boilerpipe.extract import Extractor
from collections import Counter
from newspaper import Article
from bs4 import BeautifulSoup

### concatenate bahamas dataframes / load mozambique dataframe 

In [2]:
df1 = pd.read_csv('./data/search_bahamas_20190901_to_20190904.csv')
df2 = pd.read_csv('./data/search_bahamas_20190905_to_20190908.csv')
df3 = pd.read_csv('./data/search_bahamas_20190909_to_20190912.csv')
df4 = pd.read_csv('./data/search_bahamas_20190913_to_20190916.csv')
df5 = pd.read_csv('./data/search_bahamas_20190917_to_20190920.csv')
df6 = pd.read_csv('./data/search_bahamas_20190921_to_20191001.csv')

df = pd.concat([df1, df2, df3, df4, df5, df6])

In [2]:
#df = pd.read_csv('./data/search_mozambique_20190315_to_20190414.csv')

In [3]:
len(df)

29196

### drop rows with the same URL

In [4]:
df = df.drop_duplicates(subset='SOURCEURL')

In [5]:
df.sample(10).SOURCEURL

4042    https://www.shreveporttimes.com/story/news/nat...
136     https://haitiantimes.com/2019/09/11/give-paper...
740     https://thenassauguardian.com/2019/09/04/rci-c...
5693    https://www.wuwm.com/post/after-hurricane-dori...
1839    https://www.twst.com/update/delta-air-lines-in...
2304    https://thenassauguardian.com/2019/09/25/the-p...
1960    https://reliefweb.int/report/bahamas/bahamas-h...
5423    https://reliefweb.int/report/bahamas/uk-govern...
2       https://www.kptv.com/news/local-non-profit-gro...
2742    https://www.dailymail.co.uk/tvshowbiz/article-...
Name: SOURCEURL, dtype: object

In [26]:
len(df)

9375

### crawl page (with newspaper / boilerpipe)

In [6]:
'''newspaper'''
def crawl_page(url):
    try:
        art = Article(url)
        art.download()
        art.parse()
        return art.text
    except:
        return

In [39]:
'''boilerpipe'''
def crawl_page_blplt(url):
    try:
        extr = Extractor(extractor='ArticleExtractor', url=url)
        return extr.getText()
    except:
        return

In [9]:
'''example'''
crawl_page('https://thenassauguardian.com/2019/09/04/rci-carnival-still-committed-to-development-projects/')

'0\n2369Views\nDespite not knowing the extent of the damage sustained on Grand Bahama from Hurricane Dorian,\xa0Royal Caribbean International (RCI) Chief Executive Officer Michael Bayley and\xa0Carnival Corporation\xa0Chief Executive Officer\xa0Arnold Donald reasserted their commitments to their development projects on the island.\nCarnival has already committed to the construction of a cruise port on Grand Bahama; while Holistica – a joint venture company formed by RCI and ITM Group – continues its negotiations with the government to redevelop Freeport’s cruise port and the Grand Lucayan resort complex.\nIn statements sent to the media yesterday, both RCI and Carnival also committed to helping Grand Bahama and Abaco with rebuilding efforts.\nMichael Bayley.\nHurricane Matthew caused parts of the Grand Lucayan to be shut down three years ago. The property was subsequently taken over by the government to ensure that jobs were secure and the property could be sold as quickly as possible.

In [28]:
df['article'] = df.SOURCEURL.apply(lambda url: crawl_page(url))

/home/iz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33434
  " Skipping tag %s" % (size, len(data), tag))
/home/iz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33437
  " Skipping tag %s" % (size, len(data), tag))
/home/iz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/home/iz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36868
  " Skipping tag %s" % (size, len(data), tag))
/home/iz/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly c

In [14]:
df.tail()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,article
9234,832297774,20190319,201903,2019,2019.2164,MOZ,MAPUTO,MOZ,NaN,NaN,...,4,"Maputo, Maputo, Mozambique",MZ,MZ04,-25.9653,32.5892,-319029,20190320,http://humanitariannews.org/20190320/mozambiqu...,Source: Department for International Developme...
9236,832160645,20190320,201903,2019,2019.2192,IGOEUREEC,THE EU,EUR,EEC,NaN,...,4,"Beira, Sofala, Mozambique",MZ,MZ05,-19.8436,34.8389,-307278,20190320,https://www.theguardian.com/world/2019/mar/20/...,Top story: ‘We just save what we can’\n\nHello...
9237,836021254,20190404,201904,2019,2019.2575,MOZ,MOZAMBIQUE,MOZ,NaN,NaN,...,1,Mozambique,MZ,MZ,-18.2500,35.0000,MZ,20190404,https://www.architecturaldigest.com/story/bjar...,"Policymakers, climatologists, and architects h..."
9239,835070878,20190401,201904,2019,2019.2493,AGO,ANGOLA,AGO,NaN,NaN,...,1,Mozambique,MZ,MZ,-18.2500,35.0000,MZ,20190401,https://smallwarsjournal.com/blog/russias-mili...,Russia’s Military Mission Creep Advances to a ...
9255,832450918,20190321,201903,2019,2019.2219,ZWEGOVMED,NELSON CHAMISA,ZWE,NaN,NaN,...,1,Mozambique,MZ,MZ,-18.2500,35.0000,MZ,20190321,https://bulawayo24.com/news/national/158701,News / National\n\nby Mandla Ndlovu\n\nMDC lea...


### exclude rows where article could not be extracted

In [29]:
art_df = df[df.article.notna()]

In [30]:
len(art_df)

7581

### check that articles contain one or both of the keywords

In [31]:
rel_df = art_df[art_df.article.str.contains('(?i)hurricane|dorian')]

In [32]:
len(rel_df)

6038

In [33]:
rel_df.to_csv('./data/search_bahamas_20190901_to_20191001_nwsppr.csv')